In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import math
import lasio
import os
import pandas as pd

path_to_RIGIS = r'./welldata/RIGIS/'
path_to_GIS = r'./welldata/GIS_md/'
path_to_welltracks = r'./welldata/welltracks.txt'

lasDataFrame = []
for file in os.listdir(path_to_RIGIS):
    if file.endswith('1.las'):
        lasDataFrame = (lasio.read(path_to_RIGIS+file).df())

In [ ]:
def parse_welltracks():
    welltracks = []
    f = open(path_to_welltracks)
    welldata = f.read().split(';')
    for well in welldata:
        if len(well.split()) > 2:
            name = well.split()[2][:-1]
            data = well.split()[3:]
            x = [float(data[i]) for i in range(len(data)) if i % 4 == 0]
            y = [float(data[i]) for i in range(len(data)) if i % 4 == 1]
            tvdss = [float(data[i]) for i in range(len(data)) if i % 4 == 2]
            md = [float(data[i]) for i in range(len(data)) if i % 4 == 3]
            welltracks.append([name, x, y ,tvdss, md])
    return welltracks


In [ ]:
from scipy.interpolate import interp1d
import math
welltracks = parse_welltracks()
for tracknum in range(len(welltracks)):
    f = interp1d(welltracks[tracknum][4], welltracks[tracknum][3])
    x = np.linspace(25, 900)
    plt.figure(figsize=(15,9))
    plt.plot(x, f(x))
    print(f(900))

lasDataFrame['KG_T'][lasDataFrame['KG_T'] != 0]
#lasDataFrame['KG_T'].index[0]
sum = 0
for i in range(len(lasDataFrame['KG_T']) - 1):
    if (lasDataFrame['KG_T'].array[i + 1] >= 0):
        sum += lasDataFrame['KG_T'].index[i + 1] - lasDataFrame['KG_T'].index[i]
sum